In [1]:
import pandas as pd
import numpy as np
import glob
import os
from openpyxl import load_workbook
pd.options.display.float_format = '{:,.2f}'.format

# READING FILES

### EDO DE CUENTA

In [2]:
all_mov = glob.glob(r'G:\My Drive\2.-CONCILIACIONES Y POLIZA\EDO CUENTA BANORTE\2021\**\*.txt', recursive=False)
all_mov

['G:\\My Drive\\2.-CONCILIACIONES Y POLIZA\\EDO CUENTA BANORTE\\2021\\01. ENERO\\BNTE 2021.01.04 PT1.txt',
 'G:\\My Drive\\2.-CONCILIACIONES Y POLIZA\\EDO CUENTA BANORTE\\2021\\01. ENERO\\BNTE 2021.01.04 PT2.txt',
 'G:\\My Drive\\2.-CONCILIACIONES Y POLIZA\\EDO CUENTA BANORTE\\2021\\01. ENERO\\BNTE 2021.01.04 PT3.txt',
 'G:\\My Drive\\2.-CONCILIACIONES Y POLIZA\\EDO CUENTA BANORTE\\2021\\01. ENERO\\BNTE 2021.01.04 PT4.txt',
 'G:\\My Drive\\2.-CONCILIACIONES Y POLIZA\\EDO CUENTA BANORTE\\2021\\01. ENERO\\BNTE 2021.01.04 PT5.txt',
 'G:\\My Drive\\2.-CONCILIACIONES Y POLIZA\\EDO CUENTA BANORTE\\2021\\01. ENERO\\BNTE 2021.01.04 PT6.txt',
 'G:\\My Drive\\2.-CONCILIACIONES Y POLIZA\\EDO CUENTA BANORTE\\2021\\01. ENERO\\BNTE 2021.01.04 PT7.txt',
 'G:\\My Drive\\2.-CONCILIACIONES Y POLIZA\\EDO CUENTA BANORTE\\2021\\01. ENERO\\BNTE 2021.01.04 PT8.txt',
 'G:\\My Drive\\2.-CONCILIACIONES Y POLIZA\\EDO CUENTA BANORTE\\2021\\01. ENERO\\BNTE 2021.01.04 PT9.txt',
 'G:\\My Drive\\2.-CONCILIACIONES Y P

In [3]:
edo_cta = pd.concat([pd.read_csv(f,sep='|',dtype={'Cuenta': object,'Cod. Transac':object})
                     .assign(file_name=os.path.basename(f)) for f in all_mov])
edo_cta.reset_index(inplace=True)
del edo_cta['index']
edo_cta.sort_values(by=['Movimiento'],inplace=True)
edo_cta['Cuenta'] = "'"+edo_cta['Cuenta']
edo_cta.head()

,Cuenta,Fecha De Operación,Fecha,Referencia,Descripción,Cod. Transac,Sucursal,Depósitos,Retiros,Saldo,Movimiento,Descripción Detallada,Cheque,file_name
0,'0239650517,01/01/2021,04/01/2021,701993,COMPRA ORDEN DE PAGO SPEI,511,4264,$0.00,$593.63,"$365,871,140.02",6041267,"CTA/CLABE:021320064261541207 ,CB SPEI, BCO: 02...",nan,BNTE 2021.01.04 PT1.txt
1,'0239650517,01/01/2021,04/01/2021,701994,COMPRA ORDEN DE PAGO SPEI,511,4264,$0.00,"$3,779.08","$365,867,360.94",6041268,"CTA/CLABE:014779606050609130 ,CB SPEI, BCO: 01...",nan,BNTE 2021.01.04 PT1.txt
2,'0239650517,01/01/2021,04/01/2021,701996,COMPRA ORDEN DE PAGO SPEI,511,4264,$0.00,$250.17,"$365,867,110.77",6041269,"CTA/CLABE:030320900016629283 ,CB SPEI, BCO: 03...",nan,BNTE 2021.01.04 PT1.txt
3,'0239650517,01/01/2021,04/01/2021,701989,COMPRA ORDEN DE PAGO SPEI,511,4264,$0.00,$608.48,"$365,866,502.29",6041270,"CTA/CLABE:036610500479486038 ,CB SPEI, BCO: 03...",nan,BNTE 2021.01.04 PT1.txt
4,'0239650517,01/01/2021,04/01/2021,701990,COMPRA ORDEN DE PAGO SPEI,511,4264,$0.00,$182.07,"$365,866,320.22",6041271,"CTA/CLABE:012078015430328875 ,CB SPEI, BCO: 01...",nan,BNTE 2021.01.04 PT1.txt


In [4]:
edo_cta.shape

(6246370, 14)

In [5]:
split_fecha_operacion = edo_cta["Fecha De Operación"].str.split("/",n=3,expand=True)
split_fecha_operacion.head()

,0,1,2
0,01,01,2021
1,01,01,2021
2,01,01,2021
3,01,01,2021
4,01,01,2021


In [6]:
edo_cta["Day_operacion"] = split_fecha_operacion[0] 
edo_cta["Month_operacion"] = split_fecha_operacion[1]
edo_cta['Year_operacion'] = split_fecha_operacion[2]

In [7]:
split_fecha_banco = edo_cta["Fecha"].str.split("/",n=3,expand=True)
split_fecha_banco.head()

,0,1,2
0,04,01,2021
1,04,01,2021
2,04,01,2021
3,04,01,2021
4,04,01,2021


In [8]:
edo_cta["Day_banco"] = split_fecha_banco[0] 
edo_cta["Month_banco"] = split_fecha_banco[1]
edo_cta['Year_banco'] = split_fecha_banco[2]

In [9]:
edo_cta['Fecha_operacion'] = edo_cta['Year_operacion']+'-'+edo_cta['Month_operacion']+'-'+edo_cta['Day_operacion']
edo_cta['Fecha_banco'] = edo_cta['Year_banco']+'-'+edo_cta['Month_banco']+'-'+edo_cta['Day_banco']

In [10]:
edo_cta['Fecha_operacion'] = pd.to_datetime(edo_cta['Fecha_operacion'])
edo_cta['Fecha_banco'] = pd.to_datetime(edo_cta['Fecha_banco'])

In [11]:
edo_cta.loc[edo_cta['Descripción'].str.contains('08343798'), "Tipo"] = 'XPAY'
edo_cta.loc[edo_cta['Descripción'].str.contains('08380402'), "Tipo"] = 'MIT-3DS'
edo_cta.loc[(~edo_cta['Descripción'].str.contains('08343798')) & (~edo_cta['Descripción'].str.contains('08380402')), "Tipo"] = 'REGULAR TXN'


In [12]:
edo_cta.Depósitos = [x.strip('$') for x in edo_cta.Depósitos]
edo_cta.Retiros = [x.strip('$') for x in edo_cta.Retiros]
edo_cta.Saldo = [x.strip('$') for x in edo_cta.Saldo]


In [13]:
edo_cta['Depósitos'].replace(',','', regex=True, inplace=True)
edo_cta['Retiros'].replace(',','', regex=True, inplace=True)
edo_cta['Saldo'].replace(',','', regex=True, inplace=True)

In [14]:
edo_cta['Depósitos'] = edo_cta['Depósitos'].astype(float)
edo_cta['Retiros'] = edo_cta['Retiros'].astype(float)
edo_cta['Saldo'] = edo_cta['Saldo'].astype(float)

In [15]:
edo_cta.loc[edo_cta['Cod. Transac'] == '060', 'Afiliacion_1'] = edo_cta['Descripción'].str[-9:]
edo_cta.loc[edo_cta['Cod. Transac'] == '061', 'Afiliacion_1'] = edo_cta['Descripción'].str[-9:]
edo_cta.loc[edo_cta['Cod. Transac'] == '062', 'Afiliacion_1'] = edo_cta['Descripción'].str[-9:]
edo_cta.loc[edo_cta['Cod. Transac'] == '600', 'Afiliacion_1'] = edo_cta['Descripción'].str[-9:]
edo_cta.loc[edo_cta['Cod. Transac'] == '601', 'Afiliacion_1'] = edo_cta['Descripción'].str[-9:]
edo_cta.loc[edo_cta['Cod. Transac'] == '602', 'Afiliacion_1'] = edo_cta['Descripción'].str[-9:]
edo_cta.loc[(edo_cta['Cod. Transac'] == '602') & (edo_cta['Descripción Detallada'].str.contains('CONTRACARGO')), 'Afiliacion_1'] = edo_cta['Descripción'].str[-8:]
edo_cta['len']= edo_cta['Afiliacion_1'].str.len() 
edo_cta.loc[edo_cta['len'] == 9, 'Afiliacion_final'] = edo_cta['Afiliacion_1'].str[-8:-1]
edo_cta.loc[edo_cta['len'] == 8, 'Afiliacion_final'] = edo_cta['Afiliacion_1'].str[-7:]
edo_cta.loc[edo_cta['len'] == 9, 'Tipo_operacion'] = edo_cta['Afiliacion_1'].str[-1:]
edo_cta.head()

,Cuenta,Fecha De Operación,Fecha,Referencia,Descripción,Cod. Transac,Sucursal,Depósitos,Retiros,Saldo,...,Day_banco,Month_banco,Year_banco,Fecha_operacion,Fecha_banco,Tipo,Afiliacion_1,len,Afiliacion_final,Tipo_operacion
0,'0239650517,01/01/2021,04/01/2021,701993,COMPRA ORDEN DE PAGO SPEI,511,4264,0.00,593.63,"365,871,140.02",...,04,01,2021,2021-01-01,2021-01-04,REGULAR TXN,NaN,nan,NaN,NaN
1,'0239650517,01/01/2021,04/01/2021,701994,COMPRA ORDEN DE PAGO SPEI,511,4264,0.00,"3,779.08","365,867,360.94",...,04,01,2021,2021-01-01,2021-01-04,REGULAR TXN,NaN,nan,NaN,NaN
2,'0239650517,01/01/2021,04/01/2021,701996,COMPRA ORDEN DE PAGO SPEI,511,4264,0.00,250.17,"365,867,110.77",...,04,01,2021,2021-01-01,2021-01-04,REGULAR TXN,NaN,nan,NaN,NaN
3,'0239650517,01/01/2021,04/01/2021,701989,COMPRA ORDEN DE PAGO SPEI,511,4264,0.00,608.48,"365,866,502.29",...,04,01,2021,2021-01-01,2021-01-04,REGULAR TXN,NaN,nan,NaN,NaN
4,'0239650517,01/01/2021,04/01/2021,701990,COMPRA ORDEN DE PAGO SPEI,511,4264,0.00,182.07,"365,866,320.22",...,04,01,2021,2021-01-01,2021-01-04,REGULAR TXN,NaN,nan,NaN,NaN


In [16]:
edo_cta.shape

(6246370, 27)

In [17]:
all_edo = edo_cta[['Cuenta','Fecha_operacion','Fecha_banco','Referencia','Descripción','Cod. Transac','Sucursal','Depósitos'
                   ,'Retiros','Saldo','Movimiento','Descripción Detallada','Cheque','Tipo','Afiliacion_1','Afiliacion_final'
                   ,'Tipo_operacion']]
all_edo.fillna('BLANK',inplace=True)
all_edo.head()

C:\Users\Cristian.Aparicio\anaconda3\lib\site-packages\pandas\core\frame.py:4153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


,Cuenta,Fecha_operacion,Fecha_banco,Referencia,Descripción,Cod. Transac,Sucursal,Depósitos,Retiros,Saldo,Movimiento,Descripción Detallada,Cheque,Tipo,Afiliacion_1,Afiliacion_final,Tipo_operacion
0,'0239650517,2021-01-01,2021-01-04,701993,COMPRA ORDEN DE PAGO SPEI,511,4264,0.00,593.63,"365,871,140.02",6041267,"CTA/CLABE:021320064261541207 ,CB SPEI, BCO: 02...",BLANK,REGULAR TXN,BLANK,BLANK,BLANK
1,'0239650517,2021-01-01,2021-01-04,701994,COMPRA ORDEN DE PAGO SPEI,511,4264,0.00,"3,779.08","365,867,360.94",6041268,"CTA/CLABE:014779606050609130 ,CB SPEI, BCO: 01...",BLANK,REGULAR TXN,BLANK,BLANK,BLANK
2,'0239650517,2021-01-01,2021-01-04,701996,COMPRA ORDEN DE PAGO SPEI,511,4264,0.00,250.17,"365,867,110.77",6041269,"CTA/CLABE:030320900016629283 ,CB SPEI, BCO: 03...",BLANK,REGULAR TXN,BLANK,BLANK,BLANK
3,'0239650517,2021-01-01,2021-01-04,701989,COMPRA ORDEN DE PAGO SPEI,511,4264,0.00,608.48,"365,866,502.29",6041270,"CTA/CLABE:036610500479486038 ,CB SPEI, BCO: 03...",BLANK,REGULAR TXN,BLANK,BLANK,BLANK
4,'0239650517,2021-01-01,2021-01-04,701990,COMPRA ORDEN DE PAGO SPEI,511,4264,0.00,182.07,"365,866,320.22",6041271,"CTA/CLABE:012078015430328875 ,CB SPEI, BCO: 01...",BLANK,REGULAR TXN,BLANK,BLANK,BLANK


In [18]:
all_edo.shape

(6246370, 17)

In [19]:
amex_edo_cta = all_edo[(all_edo['Descripción Detallada'].str.contains('AMERICAN EXPRESS'))]
amex_edo_cta.head()


,Cuenta,Fecha_operacion,Fecha_banco,Referencia,Descripción,Cod. Transac,Sucursal,Depósitos,Retiros,Saldo,Movimiento,Descripción Detallada,Cheque,Tipo,Afiliacion_1,Afiliacion_final,Tipo_operacion
110613,'0239650517,2021-01-04,2021-01-04,0,2021010440014 HDH0000442708310,003,5663,"414,410.14",0.00,"146,016,715.74",6151880,"SPEI RECIBIDO, BCO:0014 SANTANDER HR...",BLANK,REGULAR TXN,BLANK,BLANK,BLANK
110614,'0239650517,2021-01-04,2021-01-04,0,2021010440014 HDH0000442715480,003,5663,"7,688,275.48",0.00,"153,704,991.22",6151881,"SPEI RECIBIDO, BCO:0014 SANTANDER HR...",BLANK,REGULAR TXN,BLANK,BLANK,BLANK
111860,'0239650517,2021-01-05,2021-01-05,0,2021010540014 HDH0000455263540,003,5663,"911,226.55",0.00,"447,845,782.82",6153127,"SPEI RECIBIDO, BCO:0014 SANTANDER HR...",BLANK,REGULAR TXN,BLANK,BLANK,BLANK
111861,'0239650517,2021-01-05,2021-01-05,0,2021010540014 HDH0000455279330,003,5663,"20,508,477.51",0.00,"468,354,260.33",6153128,"SPEI RECIBIDO, BCO:0014 SANTANDER HR...",BLANK,REGULAR TXN,BLANK,BLANK,BLANK
112669,'0239650517,2021-01-06,2021-01-06,0,2021010640014 HDH0000469095340,003,5663,"668,442.03",0.00,"171,154,504.65",6153936,"SPEI RECIBIDO, BCO:0014 SANTANDER HR...",BLANK,REGULAR TXN,BLANK,BLANK,BLANK


In [20]:
amex_edo_cta.shape

(520, 17)

In [21]:
amex_edo_cta_pivot = amex_edo_cta.groupby(['Fecha_operacion','Fecha_banco']).agg({'Depósitos':sum,'Retiros':sum
                                                                      ,'Saldo':sum}).reset_index()
amex_edo_cta_pivot.head()

,Fecha_operacion,Fecha_banco,Depósitos,Retiros,Saldo
0,2021-01-04,2021-01-04,"8,102,685.62",0.00,"299,721,706.96"
1,2021-01-05,2021-01-05,"21,419,704.06",0.00,"916,200,043.15"
2,2021-01-06,2021-01-06,"7,067,021.55",0.00,"348,707,588.82"
3,2021-01-07,2021-01-07,"7,837,585.57",0.00,"328,730,543.75"
4,2021-01-08,2021-01-08,"5,263,395.91",0.00,"325,867,035.62"


In [22]:
amex_edo_cta_pivot.shape

(190, 5)

In [23]:
writer = pd.ExcelWriter('EDO CTA 2021.xlsx', engine='xlsxwriter', options={'encoding':'UTF-8'})
amex_edo_cta.to_excel(writer, sheet_name='DETALLE',index=False)
amex_edo_cta_pivot.to_excel(writer, sheet_name='BY DATE',index=False)
writer.save()
writer.close()